<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Using-2MASS-Photometry" data-toc-modified-id="Using-2MASS-Photometry-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using 2MASS Photometry</a></span></li><li><span><a href="#Using-GAIA-Measurements" data-toc-modified-id="Using-GAIA-Measurements-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Using GAIA Measurements</a></span></li></ul></div>

In [11]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import make_subplots
import pandas as pd
from uncertainties import unumpy

In [7]:
main_dir = "/u/kaimibk/Documents/Research/NAC/"
data_dir = main_dir+"data/"
out_dir = main_dir+"output/"

In [8]:
df = pd.read_csv(data_dir+"Pleiades_master.csv", index_col=0)
df.head(2)

,Name,Name.1,RA,Dec,K2MASS,(V-K)o,Period,Ref,BCK,logL/Lo,...,j_msigcom,h_m,h_msigcom,k_m,k_msigcom,iso_rad,iso_rad_Err,radius_sigma,lum_sigma,teff_sigma
0,J03394633+2358531,J03394633+2358531,54.943067,23.981442,11.83,4.89,1.138,1,2.73,-1.66,...,0.026,12.004,0.030,11.830,0.022,0.328400,0.016717,NaN,NaN,358.07666
1,J03414664+2301198,J03414664+2301198,55.444367,23.022186,13.16,5.63,0.294,1,2.81,-2.23,...,0.024,13.485,0.031,13.162,0.028,0.268599,0.009612,NaN,NaN,NaN


### Using 2MASS Photometry

Here we inspect the color magnitude diagram using the *j*, *h*, and *k* magnitudes reported by the 2MASS survey. We investigate *k* vs. *j - k* AND *j* vs. *j - h*.

In [9]:
j = unumpy.uarray(df.j_m.values, df.j_msigcom.values)
h = unumpy.uarray(df.h_m.values, df.h_msigcom.values)
k = unumpy.uarray(df.k_m.values, df.k_msigcom.values)

In [31]:
j_k_trace = go.Scatter(
    mode = "markers",
    text = df.Name,
    hoverinfo = "text+x+y",
    
    x = unumpy.nominal_values(j-k),
    y = unumpy.nominal_values(k),
    
    error_x = {
        "type" : "data",
        "array" : unumpy.std_devs(j-k),
        "visible" : True,
        "color" : 'black'
        },
    error_y = {
        "type" : "data",
        "array" : unumpy.std_devs(k),
        "visible" : True,
        "color" : 'black'
        },
    marker = {
        "color" : 'rgba(10, 100, 255, 0.3)',
        "size" : 10,
        "line" : {
            "color" : 'rgb(0, 0, 0)',
            "width" : 1
            }
        }
    )

j_h_trace = go.Scatter(
    mode = "markers",
    text = df.Name,
    hoverinfo = "text+x+y",
    
    x = unumpy.nominal_values(j-h),
    y = unumpy.nominal_values(h),
    
    error_x = {
        "type" : "data",
        "array" : unumpy.std_devs(j-h),
        "visible" : True,
        "color" : 'black'
        },
    error_y = {
        "type" : "data",
        "array" : unumpy.std_devs(j),
        "visible" : True,
        "color" : 'black'
        },
    marker = {
        "color" : 'rgba(255, 100, 15, 0.3)',
        "size" : 10,
        "line" : {
            "color" : 'rgb(0, 0, 0)',
            "width" : 1
            }
        }
    )
fig = tools.make_subplots(rows=1, cols=2)
fig.append_trace(j_k_trace, 1, 1)
fig.append_trace(j_h_trace, 1, 2)

fig['layout'].update(showlegend=False)

fig['layout']['xaxis1'].update(title='j-k', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['xaxis2'].update(title='j-h', mirror='ticks', linecolor='black', linewidth=1)

fig['layout']['yaxis1'].update(title='k', autorange = 'reversed', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['yaxis2'].update(title='j', autorange = 'reversed', mirror='ticks', linecolor='black', linewidth=1)



py.iplot(fig, filename="cmd_2mass_jk")

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



### Using GAIA Measurements

Here we inspect the color magnitude diagram using information provided by the GAIA survey, specifically *bp_rp*, color, and *phot_g_mean_mag*, apparent magnitude. Recall that apparent magnitude is converted using the following relation,

$$M = m + 5(\log_{10} p + 1)$$

In [21]:
df["gaia_abs_mag"] = df.phot_g_mean_mag + 5*np.log10(df.parallax + 1)

In [24]:
df.gaia_abs_mag.head(5)

0    15.574630
1    17.335994
2    17.800427
3    17.504567
4    15.654725
Name: gaia_abs_mag, dtype: float64

In [28]:
gaia_trace = go.Scatter(
    mode = "markers",
    text = df.Name,
    hoverinfo = "text+x+y",
    
    x = df.bp_rp,
    y = df.gaia_abs_mag,

    marker = {
        "color" : 'rgba(15, 255, 100, 0.3)',
        "size" : 10,
        "line" : {
            "color" : 'rgb(0, 0, 0)',
            "width" : 1
            }
        }
    )

layout = go.Layout(
            xaxis={
                "title" : "bp_rp",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1
            },
            yaxis={
                "title" : "M_G",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1,
                "autorange" : "reversed"
            },
    )

figure = go.Figure(data=[gaia_trace], layout=layout)

py.iplot(figure, filename="cmd_gaia")

In [34]:
fig = tools.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                          subplot_titles=('2MASS J-K','2MASS J-H', 'GAIA G Magnitude'))

fig.append_trace(j_k_trace, 1, 1)
fig.append_trace(j_h_trace, 1, 2)
fig.append_trace(gaia_trace, 2, 1)

fig['layout'].update(showlegend=False)

fig['layout']['xaxis1'].update(title='j-k', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['xaxis2'].update(title='j-h', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['xaxis3'].update(title='bp_rp', mirror='ticks', linecolor='black', linewidth=1)


fig['layout']['yaxis1'].update(title='k', autorange = 'reversed', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['yaxis2'].update(title='j', autorange = 'reversed', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['yaxis3'].update(title='M_G', autorange = 'reversed', mirror='ticks', linecolor='black', linewidth=1)


py.iplot(fig, filename='cmd_combined')

/u/kaimibk/miniconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:318: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.




This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3           -      ]

